In [ ]:
import numpy as np 
import math
from math import cos, sin, acos, asin, pi, sqrt, degrees, radians

In [ ]:
def obtainMagnitude(vector):
  vectorMag = math.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)
  return vectorMag

In [ ]:
def findQuadrant(sine, cosine):
    if cosine > 0 and sine > 0: #1
        return asin(sine)

    if cosine < 0 and sine > 0: #2
        return acos(cosine)

    if cosine < 0 and sine < 0: #3
        return pi - asin(sine)

    if cosine > 0 and sine < 0: #4
        return 2*pi + asin(sine)

In [ ]:
def obtainEcliptic(vector):
  transMatrix = [[1, 0, 0],
                 [0, cos(radians(23.4379)), sin(radians(23.4379))],
                 [0, -sin(radians(23.4379)), cos(radians(23.4379))]]

  transMatrix = np.array(transMatrix)
  vector = np.array(vector)

  ecliptic = transMatrix @ vector
  
  return ecliptic

In [ ]:
def obtainOrbitalElements(rPosition, rVelocity, time):
  rPosition = obtainEcliptic(rPosition)
  rVelocity = obtainEcliptic(rVelocity)

  rPositionMag = obtainMagnitude(rPosition)
  rVelocityMag = obtainMagnitude(rVelocity)


  #Calculate A
  a = ((2/rPositionMag) - rVelocityMag * rVelocityMag) ** - 1

  #Calculate N
  n = 0.0172020989484/math.sqrt((a**3))

  #Calculate Eccentricity
  e = math.sqrt(1 - ((obtainMagnitude(np.cross(rPosition, rVelocity))**2)/a))

  #Caclulate Inclination
  h = np.cross(rPosition, rVelocity)
  zComp = h[2]
  I = math.acos(zComp/obtainMagnitude(h))

  #Calculate Ascending Node
  hMag = obtainMagnitude(h)
  ascNodeSin = h[0]/(hMag * sin(I))
  ascNodeCos = (-h[1])/(hMag * sin(I))
  ascNode = findQuadrant(ascNodeSin, ascNodeCos)

  #Calculate Perihelion

  #5.1 
  sinFW = rPosition[2]/(rPositionMag * sin(I))
  cosFW = (1/cos(ascNode)) * ((rPosition[0]/rPositionMag) + cos(I) * sinFW * sin(ascNode))
  FW = findQuadrant(sinFW, cosFW)

  #5.2
  cosF =  (1/e) * (((a * (1-e**2))/rPositionMag) - 1)
  posVelCross = np.dot(rPosition, rVelocity)
  sinF = (posVelCross/(e*rPositionMag))*sqrt(a*(1-e**2))
  f = findQuadrant(sinF, cosF)
  w = (FW - f)
  

  # Calculate Mean Anomaly

  E = acos((1/e) * (1 - (rPositionMag/a)))

  E = 2 * pi - E

  M = E - e * sin(E)

  #Solve for T

  T = ((-M)/n) + time

  #Solve for Period

  P = ((2*pi)/n)/365.256363004

  return a, e, degrees(I), degrees(ascNode), (degrees(w) + 360), degrees(M), degrees(E), n, T, P

In [ ]:
positionVector =  [0.26617801, -1.25968549, -0.38388658]
velocityVector = [0.793688, 0.18369241, 0.38231058]
time =  2458304.74796

a, e, I, ascNode, w, M, E, n, T, P = obtainOrbitalElements(positionVector, velocityVector, time)

print("a - Semimajor axis - (au): " + str(a))
print("e - Eccentricity - (au): " + str(e))
print("i - Inclination - (degrees): " + str(I))
print("Ω - Longitude of Ascending Node - (degrees): " + str(ascNode))
print("ω - Argument of Periapsis - (degrees): " + str(w)
print("M - Mean Anomoly - (degrees): " + str(M))
print("E - Eccentric Anomoly - (degrees): " + str(E))
print("n - Mean Motion - (rad/d): " + str(n))
print("T - Time of Perehillion Passage - (JD): " + str(T))
print("P - Period - (yr): " + str(P))

a (au): 1.4732102731953043
e (au): 0.16327217193222807
i (degrees): 20.197946348367218
Ascending Node (degrees): 263.8617162759592
Mean Anomoly (degrees): 310.50791873543153
E (degrees): 302.6295391907052
n (rad/d): 0.009620205150144227
T: 2457741.4142835485
P (yr): 1.7881244623209407
84.36990981479016
